# Basics of Clustering Methods for Linguistic Data

## Society for Linguistics Undergrad Students (SLUgS) March 31, 2022 - Eric Wilbanks


## 1) Overview

- who is this for?
    - folks interested in language data
    - no experience with statistics or coding required (though basic Python knowledge will help you understand the examples)
    
- what are we doing?
    - motivation of clustering methods
    - basic types of clustering approaches and their uses
    - example python code and hands-on activities
    
- what do you need to get started?
    - review the README.md file
    - consider using Datahub (or your own local Jupyter installation)

In [ ]:
# load dependencies
from urllib import request # for downloading
import pandas as pd # for data manipulations
pd.options.mode.chained_assignment = None  # default='warn'; since we're only doing column additions in place, this is fine. Use at your own peril though...
import numpy as np # for everything really
import matplotlib as mpl # for plotting
import seaborn as sns # for plotting
from sklearn import cluster, mixture # clustering methods
from scipy import linalg, stats
import itertools

## 2) Clustering Basics

**Why use them? Why do we care about clusters?**

- linguistic data are hierarchically structured, e.g.:
    - individual vowel utterances are grouped within phonemes
    - sentences are grouped under authors
    - languages are grouped under language families

- we might go in with hypotheses about the structure of the system, or we might want to discover this structure.
- in any case, clustering algorithms are one approach to quantifying this structure.

- we'll mainly be looking at 2D and 1D clustering, but most methods scale to higher order problems as well. 

Clustering Methods are very diverse, but we can characterize many of them by how they go about finding clusters:
- ***Centroids***: picking a central cluster point and assigning points to clusters depending on their distance to this centroid (e.g., k-means)
- ***Distributions***: fitting probability distributions to categories and using those to get a measure of each data point's likely category assignment (e.g., Gaussian Mixture Models)
- ***Neighborhood Densities***: like centroid-based methods, density algorithms look at the relationship between points, but take into account the density of adjacent neighbors. This allows for less influence of outliers and better identification of core clusters. (e.g., DBSCAN)
- ***Hierarchical Structures***: define clusters on the basis of connection between data points (e.g., dendrograms)

When you're choosing a clustering algorithm for your task/data, you'll need to consider questions such as:

- Will all the clusters have the same number/density of points?
- Will they be the same shape? The same size?
- How variable will they be? How many outliers are in the data?
- How many data points will I have?
- Can a data point be members of multiple clusters (fuzzy/soft clustering) or is it only ever in one cluster?

We're going to start with some basic algorithms today, since deciding between the various options is beyond the scope of this workshop!

## 3) K-Means Clustering : Vowel Measurements



In [ ]:
# download the csv of vowel measurements
spade_buckeye_url = 'https://osf.io/c5u7a/download'
spade_buckeye_csv = './spade_buckeye.csv'
request.urlretrieve(spade_buckeye_url, spade_buckeye_csv)
# read in as pandas dataframe
vowels = pd.read_csv(spade_buckeye_csv)
# remove rows where we don't have F1 measurements
vowels = vowels[~vowels['F1'].isnull()]

### 3.1 Look at the data!

Your very first step should always be to take a look at the data. 

What are we dealing with? What are the variables we have? How many speakers are in this dataset? What do the vowel measurements look like? What are these non-IPA symbols? (hint: it's lower-case [Arpabet](https://en.wikipedia.org/wiki/ARPABET))

In [ ]:
# variables in our set
vowels.columns

In [ ]:
# get number of observations per-speaker
print(vowels.groupby('speaker').size())

In [ ]:
# now per-phone
print(vowels.groupby('phone_label').size())

Vowel formant measurements vary systematically by speakers, in part due to differences in vocal-tract length.

Ideally, we want to compare between-speaker measurements after normalizing for these differences.

There are a lot of options, but a useful first pass method is **z-score normalization**.
- zscore transform all measurements of a given speaker

In [ ]:
# grouping by speaker and then zscoring F2 and F1
zscores = vowels.groupby('speaker')[['F2','F1']].transform(stats.zscore).rename(columns={'F2':'F2.z', 'F1':'F1.z'})
# append new columns
vowels = pd.concat([vowels, zscores], axis=1)

In [ ]:
# define our phone_labels for a subset of vowels
subset_vowels = ['iy','aa','uw']
# subset vowels to just be in the subset
subset = vowels[vowels['phone_label'].isin(subset_vowels)]
# plot F1 and F2: standard and z-score
fig, (ax1,ax2) = mpl.pyplot.subplots(1,2)
# individual scatterplots
sns.scatterplot(x='F2', y='F1', data=subset, hue='phone_label', ax=ax1)
sns.scatterplot(x='F2.z', y='F1.z', data=subset, hue='phone_label', ax=ax2)
# titles
ax1.set_title('Standard Hz')
ax2.set_title('Z-Scored')
# flip axes for typical orientation
ax1.invert_xaxis() 
ax1.invert_yaxis()
ax2.invert_xaxis()
ax2.invert_yaxis()
# spacing
fig.tight_layout()
# show plot
mpl.pyplot.show()

### 3.2 k-means algorithm basics

k-means clustering is a broad family of related algorithms that are all ***centroid*** based methods. 

The basic process is:
1. Randomly assign all data points to k number of clusters
2. Calculate the mean/center of each cluster
3. Reassign each data point to the nearest cluster center
4. Repeat 2-3 until we converge on a stable assignment

In [ ]:
# pull out just the F2.z and F1.z columns
formants = subset.loc[:,['F2.z','F1.z']]
# specify number of clusters
k_subset_zscore = cluster.KMeans(n_clusters = 3).fit(formants)
# add identified cluster labels to dataframe
subset['cluster_label'] = k_subset_zscore.labels_

In [ ]:
# plot F2.z and F1.z against predicted cluster labels
ax = sns.scatterplot(x='F2.z', y='F1.z', data=subset, hue='cluster_label')
center_x, center_y = zip(*k_subset_zscore.cluster_centers_) # zip(*) to convert [[x1,y1],[x2,y2] to [(x1,x2),(y1,y2)]
ax.scatter(center_x, center_y, marker='P', color='purple', s=100)
ax.invert_xaxis() # flip axis for typical orientation
ax.invert_yaxis()
mpl.pyplot.show()

In [ ]:
# visualize crosstab percentages
pd.crosstab(subset['phone_label'], subset['cluster_label'], normalize='index')

In [ ]:
# plot F2.z and F1.z against manual and automatic clustering
ax = sns.scatterplot(x='F2.z', y='F1.z', data=subset, hue=subset[['phone_label', 'cluster_label']].apply(tuple, axis=1))
ax.scatter(center_x, center_y, marker='P', color='purple', s=100)
ax.invert_xaxis() # flip axis for typical orientation
ax.invert_yaxis()
mpl.pyplot.show()

### 3.3 Extension - Unnormalized

Now, let's do the same analysis on the unnormalized formant measurements. How does the automatic clustering compare to the clustering on the z-scored measurements?

In [ ]:
# pull out just the F2 and F1 columns
#formants_un = 

# specify number of clusters
#k_subset_un = 

# add identified cluster labels to dataframe
#subset['cluster_label_un'] = 
#center_x, center_y = zip(*k_subset_un.cluster_centers_) # zip(*) to convert [[x1,y1],[x2,y2] to [(x1,x2),(y1,y2)]

# plot F2.z and F1.z against manual and automatic clustering
#ax = sns.scatterplot(x='F2', y='F1', data=subset, hue=subset[[??]].apply(tuple, axis=1))
#ax.scatter(center_x, center_y, marker='P', color='purple', s=100)
#ax.invert_xaxis() # flip axis for typical orientation
#ax.invert_yaxis()
#mpl.pyplot.show()

In [ ]:
# visualize crosstab percentages
#print(pd.crosstab(subset['phone_label'], subset['cluster_label_un'], normalize='index'))
#print()
# visualize normalized percentages to compare
#print(pd.crosstab(subset['phone_label'], subset['cluster_label'], normalize='index'))

## 4. How many clusters do I need?

Depending on your question and domain, you might not want to/be able to specify the number of clusters a priori.

One method of evaluating the appropriate number of clusters is the ***elbow method***.

More clusters will always explain more of the variance in the data, but at a certain point the increased variance explained for each cluster will become negligible. We want to find the "elbow" in the graph of number of clusters vs. variance explained, to optimize for predictiveness with the fewest number of clusters.

sklearn.kmeans has a built-in implementation of "inertia" (within-category sum-of-squares)

In [ ]:
inertias = {} # container for inertia
for k in range(1,7):
    # specify number of clusters
    k_subset_zscore = cluster.KMeans(n_clusters = k).fit(formants)
    inertias[k] = k_subset_zscore.inertia_

In [ ]:
mpl.pyplot.plot(inertias.keys(), inertias.values())
mpl.pyplot.show()

In this case, the elbow-method points to k=2 to be the likely best number of clusters.

However, with our expert linguistic knowledge we're sure that the "correct" number of clusters is 3. 

What's going on here?

In [ ]:
# specify number of clusters
k_subset_z2 = cluster.KMeans(n_clusters = 2).fit(formants)
# add identified cluster labels to dataframe
subset['cluster_label_z2'] = k_subset_z2.labels_
center_x, center_y = zip(*k_subset_z2.cluster_centers_) # zip(*) to convert [[x1,y1],[x2,y2] to [(x1,x2),(y1,y2)]
# plot F2.z and F1.z against manual and automatic clustering
ax = sns.scatterplot(x='F2.z', y='F1.z', data=subset, hue=subset[['phone_label', 'cluster_label_z2']].apply(tuple, axis=1))
ax.scatter(center_x, center_y, marker='P', color='purple', s=100)
ax.invert_xaxis() # flip axis for typical orientation
ax.invert_yaxis()
mpl.pyplot.show()

In [ ]:
pd.crosstab(subset['phone_label'], subset['cluster_label_z2'], normalize='index')

## 4.1 Extension - tense vowels

In [ ]:
# define our phone_labels for the tense vowels
tense_vowels = ['aa','ey','iy','ow','uw']
# subset vowels to just be in the tense set
tense = vowels[vowels['phone_label'].isin(tense_vowels)]
# plot F1 and F2
ax = sns.scatterplot(x='F2.z', y='F1.z', data=tense, hue='phone_label')
ax.invert_xaxis() # flip axis for typical orientation
ax.invert_yaxis()
mpl.pyplot.show()

Now, using the `tense` dataset above and use the k-means clustering algorithm with k=5.

Then, visualize the cluster assignments using a plot like we've seen before.

What types of clusters do we end up with? Are they linguistically reasonable?

In [ ]:
#tense_formants_z = ??
#k_tense_z = ??

In [ ]:
#tense['cluster_label'] = k_tense_z.labels_

In [ ]:
# plot F1 and F2
#ax = sns.scatterplot(x='F2.z', y='F1.z', data=tense, hue='cluster_label')
#center_x, center_y = zip(*k_tense_z.cluster_centers_)
#ax.scatter(center_x, center_y, marker='P', color='red', s=100)
#ax.invert_xaxis() # flip axis for typical orientation
#ax.invert_yaxis()
#mpl.pyplot.show()

# 5. GMM


Let's consider a different clustering method which focuses on maximizing probability and fitting distributions to the data: Gaussian Mixture Models (GMMs).

This approach is useful especially when you have some reason to believe that your data-generating process might involve Gaussian (normal) distributions (such as if speakers have a shared acoustic target and (normally-distributed) noise is introduced during vowel production.

In [ ]:
tense_formants_z = tense.loc[:,['F2.z','F1.z']]
# initialize a GMM with 5 components/clusters
g_tense = mixture.GaussianMixture(n_components=5)
# fit this GMM to the data
g_tense.fit(tense_formants_z)
# pull out the most likely category label and save as a df colun
tense['gmm_label'] = g_tense.predict(tense_formants_z)
tense = tense.reset_index(drop=True)

# use our category models to predict the probability of each data point being classified as each cluster
probs = pd.DataFrame(g_tense.predict_proba(tense_formants_z)).rename(columns={0:'prob_0',1:'prob_1',2:'prob_2',3:'prob_3',4:'prob_4'})
tense = pd.concat([tense,probs], axis=1)

In [ ]:
# what does `probs` look like?
probs

In [ ]:
def plot_ellipses(ax,fit):
    # adapted from http://scikit-learn.org/stable/auto_examples/mixture/plot_gmm.html
    for i, el in enumerate(fit.means_):
        # calculate ellipse parameters
        v, w = linalg.eigh(fit.covariances_[i])
        v = 2.0 * np.sqrt(2.0) * np.sqrt(v)
        u = w[0] / linalg.norm(w[0])
        
        # Plot an ellipse to show the Gaussian component
        angle = np.arctan(u[1] / u[0])
        angle = 180.0 * angle / np.pi  # convert to degrees
        p = mpl.patches.Ellipse(fit.means_[i], v[0], v[1], 180.0 + angle, color='black')
        p.set_alpha(0.5)
        ax.add_artist(p)

In [ ]:
# plot F1 and F2
ax = sns.scatterplot(x='F2.z', y='F1.z', data=tense, hue = 'gmm_label', palette = 'colorblind')
plot_ellipses(ax,g_tense)
ax.invert_xaxis() # flip axis for typical orientation
ax.invert_yaxis()
mpl.pyplot.show()

# plot F1 and F2
ax = sns.scatterplot(x='F2.z', y='F1.z', data=tense, hue = 'phone_label', palette = 'colorblind')
plot_ellipses(ax,g_tense)
ax.invert_xaxis() # flip axis for typical orientation
ax.invert_yaxis()
mpl.pyplot.show()

In [ ]:
pd.crosstab(tense['phone_label'], tense['gmm_label'], normalize='index')

What's going on here?

We have a lot of data with a ton of variance. Some of that token-level variance is likely due to difference in phonological context, perhaps lexical factors, local speaking rate, individual speakers differing in linguistic systems, etc. 

This makes the clustering task extremely difficult, if the clustering methods are trying to accurately predict ALL the observed data.

Let's consider what happens when we just look at speaker-means, averaging over some of these other influential factors:

In [ ]:
# calculate by-speaker means
tense_means = tense.groupby(['speaker','phone_label'])[['F2.z','F1.z']].mean().reset_index()
tense_means

In [ ]:
# fit a 5 cluster GMM to the tense_means df
tense_formants_mean = tense_means.loc[:,['F2.z','F1.z']]
g_tense_means = mixture.GaussianMixture(n_components=5)
g_tense_means.fit(tense_formants_mean)
tense_means['gmm_label'] = g_tense_means.predict(tense_formants_mean)

In [ ]:
# plot F1 and F2
ax = sns.scatterplot(x='F2.z', y='F1.z', data=tense_means, hue = 'gmm_label', palette = 'colorblind')
plot_ellipses(ax,g_tense_means)
ax.invert_xaxis() # flip axis for typical orientation
ax.invert_yaxis()
mpl.pyplot.show()

In [ ]:
# compare ground-truth linguist classifications with cluster assignments 
# (note the results for k-means on a speaker-mean df are basically the same)
pd.crosstab(tense_means['phone_label'], tense_means['gmm_label'], normalize='index')

In [ ]:
aics = {} # container for AICs
for k in range(1,10):
    # specify number of clusters
    g = mixture.GaussianMixture(n_components=k)
    g.fit(tense_formants_z)
    aics[k] = g.aic(tense_formants_z)

mpl.pyplot.plot(aics.keys(), aics.values())
mpl.pyplot.show()

# 5.1 Extension - GMM for tense_means with 2 clusters

You might be able to guess what the two likely clusters as noted by the elbow method would be. Briefly in this extension, carry out the analysis and visualize the clusters.

In [ ]:
# 2 cluster GMM
#g_tense_means2 = ??
#g_tense_means2.fit(tense_formants_mean)
#tense_means['gmm_label_2'] = ??

In [ ]:
# plot F1 and F2
#ax = sns.scatterplot(??)
#plot_ellipses(ax,g_tense_means2)
#ax.invert_xaxis() # flip axis for typical orientation
#ax.invert_yaxis()
#mpl.pyplot.show()

# 6. DBSCAN

DBSCAN is a density based clustering method that attempts to locate core regions and ignore outliers by considering each point's neighbors and how dense its local region is. 

Unlike the methods we've seen so far, DBSCAN automatically choose the optimal number of components/clusters based on the parameters we set and the data.

In [ ]:
# initialize model object
tense_means_db = cluster.DBSCAN()
# fit to data (just the F2 and F1 tense formants
tense_means_db.fit(tense_formants_mean)
# save cluster labels
tense_means['db_labels'] = tense_means_db.labels_

In [ ]:
# plot F1 and F2
ax = sns.scatterplot(x='F2.z', y='F1.z', data=tense_means, hue = 'db_labels', palette = 'colorblind')
ax.invert_xaxis() # flip axis for typical orientation
ax.invert_yaxis()
mpl.pyplot.show()

The most important parameter of the DBSCAN method is `eps`: short for epsilon, which here represents the maximum distance between two samples for them to be considered a neighbor.

The default is 0.5. What happens when we set it to 0.2?

In [ ]:
# initialize model object
#tense_means_db_eps02 = ??
# fit to data (just the F2 and F1 tense formants
#??
# save cluster labels
#tense_means['db_labels_eps02'] = ??

In [ ]:
# plot F1 and F2
#ax = sns.scatterplot(x=??, y=??, data=tense_means, hue = ??, palette = 'colorblind')
#ax.invert_xaxis() # flip axis for typical orientation
#ax.invert_yaxis()
#mpl.pyplot.show()